# Deep Learning - Exercise 7

This lecture is focused on build own unsupervised word embedding using Word2Vec Skip-Gram method.

We will use the embedding for text generation using RNN.

[Open in Google colab](https://colab.research.google.com/github/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_06.ipynb)
[Download from Github](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_06.ipynb)

##### Remember to set **GPU** runtime in Colab!

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow import string as tf_string
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

from sklearn.model_selection import train_test_split # 
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import normalize
import scipy
import itertools
import string
import re
import tqdm
import io

tf.version.VERSION

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

In [ ]:
SEED = 13

In [ ]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

In [ ]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

In [ ]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

In [ ]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

In [ ]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

In [ ]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

In [ ]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

In [ ]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word

In [ ]:
context

In [ ]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

In [ ]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file = 'datasets/hp1.txt'

In [ ]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

In [ ]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [ ]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))

In [ ]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

In [ ]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

In [ ]:
targets

In [ ]:
contexts

In [ ]:
labels

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

In [ ]:
# https://stackoverflow.com/questions/26089893/understanding-numpys-einsum

In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = keras.layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = keras.layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
history = word2vec.fit(dataset, epochs=20)

In [ ]:
show_history(history)

In [ ]:
word2vec.summary()

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
inverse_vocab[:20]

In [ ]:
id2word = {k: v for k, v in enumerate(vocab)}
word2id = {v: k for k, v in enumerate(vocab)}

In [ ]:
inverse_vocab[20:40]

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

distance_matrix = cosine_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['god', 'jesus', 'noah', 'egypt', 'john', 'gospel', 'moses','famine']}

similar_words

In [ ]:
from sklearn.manifold import TSNE

words = sum([[k] + v for k, v in similar_words.items()], [])
words_ids = [word2id[w] for w in words]
word_vectors = np.array([weights[idx] for idx in words_ids])
print('Total words:', len(words), '\tWord Embedding shapes:', word_vectors.shape)

tsne = TSNE(n_components=2, random_state=0, n_iter=10000, perplexity=3)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(word_vectors)
labels = words

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue', edgecolors='k')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

In [ ]:
with open('datasets/bible.txt') as f:
    lines = f.readlines()

In [ ]:
lines = [x.replace('\n', '') for x in lines if x != '\n']

In [ ]:
norm_bible = ''.join(lines)

In [ ]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
vectorize_layer.adapt(lines)

In [ ]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

In [ ]:
id2word = {k: v for k, v in enumerate(inverse_vocab)}
word2id = {v: k for k, v in enumerate(inverse_vocab)}

In [ ]:
vectorize_layer('jesus god').numpy()

In [ ]:
wids = [[w for w in vectorize_layer(doc).numpy()] for doc in lines]
print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

In [ ]:
from keras.preprocessing.sequence import skipgrams

# generate skip-grams
skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in wids]

# view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id2word[pairs[i][0]], pairs[i][0], 
          id2word[pairs[i][1]], pairs[i][1], 
          labels[i]))

In [ ]:
from tensorflow.keras.layers import Concatenate, Dot
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential

# build skip-gram architecture
word_model = Sequential()
word_model.add(Embedding(vocab_size, embed_size,
                         embeddings_initializer="glorot_uniform",
                         input_length=1))
word_model.add(Reshape((embed_size, )))

context_model = Sequential()
context_model.add(Embedding(vocab_size, embed_size,
                  embeddings_initializer="glorot_uniform",
                  input_length=1))
context_model.add(Reshape((embed_size,)))

model = Sequential()
model.add(Dot(axes=(1, 2))([word_model, context_model]))
model.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))
model.compile(loss="mean_squared_error", optimizer="rmsprop")

# view model summary
print(model.summary())


# 🔎 We will work with LSTM/GRU layers mostly 🔎
* Do you know anything abour RNN in general?
    * How are they different from FCNN?

![Meme02](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_06_meme_02.png?raw=true)



* How is pure RNN and LSTM/GRU layer different?
    * What issue of RNN do they address?
* Can you imagine some use-cases for RNN?
    * Can you imagine some limits of ML/DL solutions in the usecases as well?


## 🔎 We have some new packages today 🔎
* Below is a short description of them, check out the URLs for more details and API 

### NLTK
* NLTK is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, wrappers for industrial-strength NLP libraries, and an active discussion forum.
    * https://www.nltk.org/

### TextBlob
* TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.
    * https://textblob.readthedocs.io/en/dev/

In [ ]:
import unicodedata, re, string
import nltk
from textblob import TextBlob

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

## Punkt Sentence Tokenizer

* This tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences. It must be trained on a large collection of plaintext in the target language before it can be used.

In [ ]:
nltk.download('punkt')

## Download the dataset

In [ ]:
df = pd.read_csv('https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/raw/main/datasets/train_tweets.csv')

# Let's take a look at the data

In [ ]:
df.head()

In [ ]:
df.shape

## We can see that the classification task is highly imbalanced, because we have only 2242 negative tweets compared with positive one

In [ ]:
sns.countplot(x='label', data=df)

In [ ]:
df.label.value_counts()

In [ ]:
df['length'] = df.tweet.apply(len)

### We can see that the sentences are of similar lengths

In [ ]:
sns.boxplot(x='label', y='length', data = df)

## We can see that the text data are full of noise

* 💡 Social posts suffer the most from this effect
    * The text is full of hashtags, emojis, @mentions and so on
* These parts usually don't influence the sentiment score by much
* Although most advanced models usually extract even this features because e.g. emojis can help you with the sarcasm understanding

## Take a look at few examples, it will share many of these caveates which we discussed for sure

In [ ]:
for x in df.loc[:10, 'tweet']:
    print(x)
    print('---------')

# 💡Text data pre-processing uses a few specific techniques
* Benefit of using these techniques varies from approach to approach
* However it is good to have some knowledge about them

## Stemming
* Stemming is the process of producing morphological variants of a root/base word. 
    * Stemming programs are commonly referred to as stemming algorithms or stemmers. 
* A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”

#### 💡 Examples of stemming:
* chocolates, chocolatey, choco : chocolate
* retrieval, retrieved, retrieves : retrieve


## Lemmatization 
* Lemmatization is the process of grouping together the different inflected forms of a word so they can be analysed as a single item. Lemmatization is similar to stemming but it brings context to the words. So it links words with similar meaning to one word.

#### 💡 Examples of lemmatization:
* rocks : rock
* corpora : corpus
* better : good

## Both techiques can be used in the preprocessing pipeline
* You have to decide if it is beneficial to you, because this steps leads to some generalization of the data by itself. 
    * You will definitely lose some pieces of the information. 

# 💡Embedding tip
* If you use some form of embedding like Word2Vec or Glove, it is better to skip this steps because the embedding vocabulary skipped it as well. 🙂

## You don't have to code the pre-process steps yourself
* We have already prepared the most common functions used
* Modify function *normalize()* for different steps

In [ ]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_numbers(words):
    """Remove all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        new_word = re.sub("\d+", "", word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
# words = remove_punctuation(words)
    words = remove_numbers(words)
#    words = remove_stopwords(words)
    return words

def form_sentence(tweet):
    tweet_blob = TextBlob(tweet)
    return tweet_blob.words

# Tokenize sentences and remove puncuation by TextBlob library

In [ ]:
df['Words'] = df['tweet'].apply(form_sentence)

In [ ]:
df.head()

# Normalize sentences 
* We want only ascii and lowercase text and no numbers in the strings

## You can experiments with different preprocess steps as you wish! 🙂

In [ ]:
df['Words_normalized'] = df['Words'].apply(normalize)

In [ ]:
df.head()

## Remove the 'user' word from tweets

In [ ]:
df['Words_normalized_no_user'] = df['Words_normalized'].apply(lambda x: [y for y in x if 'user' not in y])

In [ ]:
df.head()

## We can see that no pre-processing is ideal and we have to fix some issues by ourselves
* e.g. n't splitting

In [ ]:
print(df.tweet.iloc[1])
print(df.Words_normalized_no_user.iloc[1])

In [ ]:
def fix_nt(words):
    st_res = []
    for i in range(0, len(words) - 1):
        if words[i+1] == "n't" or words[i+1] == "nt":
            st_res.append(words[i]+("n't"))
        else:
            if words[i] != "n't" and words[i] != "nt":
                st_res.append(words[i])
    return st_res

In [ ]:
df['Words_normalized_no_user_fixed'] = df['Words_normalized_no_user'].apply(fix_nt)

## The issue is now fixed

In [ ]:
print(df.tweet.iloc[1])
print(df.Words_normalized_no_user.iloc[1])
print(df.Words_normalized_no_user_fixed.iloc[1])

## We can join the text into single string for each instance

In [ ]:
df['Clean_text'] = df['Words_normalized_no_user_fixed'].apply(lambda x: " ".join(x))

In [ ]:
df['Clean_text'].head()

# Let's take a look at the most common words in corpus
* It is one of the usual EDA step for text data
* Although without any preprocessing there will be a lot of so-called *stopwords*

## 🔎 Do you know what the term **stopword** mean?

## Just tokenize every string and merge the token array into one big array using *itertools.chain()*

In [ ]:
all_words = list(itertools.chain(*df.Words_normalized_no_user_fixed))

In [ ]:
all_words[:10]

## Compute frequency of every token using *nltk*

In [ ]:
dist = nltk.FreqDist(all_words)

## The most common tokens are:

In [ ]:
dist

## We have 34289 unique words (tokens)

In [ ]:
len(dist)

## The longest tweet has 42 words

In [ ]:
max(df.Words_normalized_no_user_fixed.apply(len))

# Our dataset is ready, we can start our Deep learning experiments 🚀

## Can you use regular FCANN for the sentiment analysis?

![Meme01](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_06_meme_01.png?raw=true)


# We will use TextVectorization layer for creating vector model from our text data
* For those of you who are interested in the topic there is very good [article on Medium](https://towardsdatascience.com/you-should-try-the-new-tensorflows-textvectorization-layer-a80b3c6b00ee) about the layer and its parameters.

* There is of course a [documentation page](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization) about the layer as well.

## What does Text vectorization mean in this context?
* Is it any different term from the one used in information retrieval?

## There is a few important parameters:
* emedding_dim 
    * Dimension of embedded representation
    * This is already part of latent space, there is captured dependecy among words in these vectors, we are learning this vectors using the ANN
* vocab_size
    * Number of unique tokens in vocabulary
* sequence_length
    * Output dimension after vectorizing - words in vectorited representation are independent

In [ ]:
embedding_dim = 128 # Dimension of embedded representation - this is already part of latent space, there is captured some dependecy among words, we are learning this vectors in ANN
vocab_size = 10000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing - words in vectorited representation are independent

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(df.Clean_text.values)

## We will split our dataset to train and test parts with stratification

## ⚠⚠ COMPETITION TEST SET HERE ⚠⚠

### COMPETITION ?!?! 🤔
* I will provide the details in the end of the lecture 🙂

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.Clean_text, df.label, test_size=0.20, random_state=13, stratify=df.label)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=13, stratify=y_train)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
print('Train')
print(y_train.value_counts())
print('Test')
print(y_test.value_counts())

In [ ]:
print('Vocabulary example: ', vect_layer.get_vocabulary()[:10])
print('Vocabulary shape: ', len(vect_layer.get_vocabulary()))

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer=keras.optimizers.RMSprop(), loss=keras.losses.BinaryCrossentropy(), metrics=keras.metrics.BinaryAccuracy())

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.tf',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

In [ ]:
batch_size = 128
epochs = 5

history = model.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), callbacks=[model_checkpoint_callback], epochs=epochs, batch_size=batch_size)

show_history(history)

In [ ]:
# Load best setup
model.load_weights("weights.best.tf")

In [ ]:
y_pred = model.predict(X_test).ravel()

## Sigmoid function gives us real number in range <0, 1>.

In [ ]:
y_pred[:10]

## We need to map this values to discreet classes 0 and 1

In [ ]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

In [ ]:
y_pred[:10]

# We can see that accuracy is not the best metric in the imbalanced situation - you already know the reason 🙂
* There are many more metrics we can use and one of the most common in this situation is the F1 Score, see [this](https://en.wikipedia.org/wiki/F-score) and [this](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/) for more info

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_pred)

In [ ]:
f1_score(y_true=y_test, y_pred=y_pred)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_true=y_test, y_pred=y_pred), annot=True, cmap='Greens', fmt='.0f')

# Do we need to train our own embedding from scratch? 🤔
* There are multiple embeddings available online which were trained on very large corpuses e.g. Wikipedia. 
* Good examples are Word2Vec, Glove or FastText. 
    * These embeddings contains fixed length vectors for words in the vocabulary.

* We will use GloVe embedding with 50 dimensional embedding vectors. For more details see [this](https://nlp.stanford.edu/projects/glove/).
* You can download zip with vectors from [http://nlp.stanford.edu/data/glove.6B.zip](http://nlp.stanford.edu/data/glove.6B.zip) ~ 800 MB

# ⚠ Beware that the original text corpus was more general than the specific social media text data
* So if you deal with very specific domains it may be beneficial to train your own embedding or at least fine tune existing one in the end

# We need to download the embedding files
~~~
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
~~~

50 dims GLOVE is also avaiable here: https://vsb.ai/downloads/glove.6B.50d.txt

# First we need to load the file to memory and create embedding dictionary

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [ ]:
path_to_glove_file = 'glove.6B.50d.txt'

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

## 💡 This is how the embedding latent vector looks like for the word 'analysis'

In [ ]:
embeddings_index['analysis']

In [ ]:
embeddings_index['analysis'].shape

## We need to get the voacabulary from the Vectorizer and the integer indexes

In [ ]:
embedding_dim = 50 # Embedding dimension -> GloVe 50
vocab_size = 10000 # Number of unique tokens in vocabulary
sequence_length = 20 # Output dimension after vectorizing - words in vectorited representation are independent

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(df.Clean_text.values)

In [ ]:
voc = vect_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
voc[:10]

In [ ]:
word_index['the']

In [ ]:
embeddings_index['the']

# Map the *int* indices to the embedding vectors and save the mapping to the matrix
* Every row is one token

In [ ]:
num_tokens = len(voc) + 2
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
embedding_matrix[2]

# Use the GloVe embedding in the Embedding layer in your own model
* Beware the *embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False*
    * You say the model to use the GloVe vectors and that it can't modify them
    * You can of course set *trainable=True* and do the fine-tuning

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer=keras.optimizers.RMSprop(), loss=keras.losses.BinaryCrossentropy(), metrics=keras.metrics.BinaryAccuracy())

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.tf',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

In [ ]:
batch_size = 128
epochs = 5

history = model.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), callbacks=[model_checkpoint_callback], epochs=epochs, batch_size=batch_size)

show_history(history)

In [ ]:
# Load best setup
model.load_weights("weights.best.tf")

# Which model is better?

In [ ]:
y_pred = model.predict(X_test).ravel()
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
print(f'Accuracy: {accuracy_score(y_true=y_test, y_pred=y_pred)}')
print(f'F1 Score: {f1_score(y_true=y_test, y_pred=y_pred)}')

## Tasks for the lecture (2p)

* Try to create your own architecture (1p)
    * Experiment a little - try different batch sizes, optimimizers, time lags as features, etc.

* Try to fine-tune the GloVe embedding or use GloVe 100 and compare the models with the non-trainable one (1p)
    * Is it any different according to the F1-Score?

# There is a competition for bonus points this week! 🚀
- Everyone who will send me a correct solution will be included in the F1 - Score toplist
- Deadline for the competition submission is Monday 10th at 23:59
- The toplist will be publicly available on Wednesday
- There is no limitation in used layers (LSTM, CNN, ...), optimizers, etc. 
- You can use any model architecture from the internet including transfer learning




## The only limitation is that the model has to be trained/fine-tuned on Colab/Kaggle/Your machine so online sentiment scoring services are forbidden!

![Meme03](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_06_meme_03.png?raw=true)


# The winner with the best F1 - Score on test set will be awarded with 5 bonus points

## ⚠ The test set is the same as we used in the lecture ⚠
* It is marked with ⚠⚠ in the notebook